In [1]:
!pip install geopandas shapely pykml


In [2]:
from pykml import parser
from shapely.geometry import Polygon, Point
from shapely.ops import nearest_points



In [3]:
def get_extremity_point(polygon1, polygon2, polygons_z):
    """Return the point in polygon1 that is nearest to polygon2."""
    nearest = nearest_points(polygon1, polygon2)
    for polygon_z in polygons_z:
        for point_z in polygon_z:
            if (point_z[0] == nearest[0].x) and (point_z[1] == nearest[0].y):
                return point_z

In [4]:
# Parse the KML file
with open(r'C:\Users\Admin\Downloads\Data\Data\Site_UAE_v2.kml', 'r') as kml_file: #change the path as the kml file_path
    doc = parser.parse(kml_file).getroot().Document

In [5]:
# Extract the polygons from the KML file
polygons = []
polygons_z = []
for placemark in doc.Placemark:
    coordinates_text = placemark.Polygon.outerBoundaryIs.LinearRing.coordinates.text
    coordinates = [(float(x), float(y)) for x, y, *z in (pt.split(',') for pt in coordinates_text.split())]
    coordinates_z = [(float(x), float(y), float(z[0]) if z else 0) for x, y, *z in (pt.split(',') for pt in coordinates_text.split())]
    polygons.append(Polygon(coordinates))
    polygons_z.append(coordinates_z)

In [6]:
# Get the extremity point for each pair of polygons
substations = [get_extremity_point(polygons[i], polygons[i + 1], polygons_z) for i in range(len(polygons) - 1)]


In [7]:
# If there is only one polygon, find the point on it with the highest altitude
if len(polygons) == 1:
    substations.append(max(polygons_z[0], key=lambda x: x[2]))

In [8]:
# Print the substation positions
for i, point in enumerate(substations, 1):
    x, y, z = point
    print(f'Substation {i}: Longitude: {x}, Latitude: {y}, Altitude: {z}')

Substation 1: Longitude: -110.12365948868883, Latitude: 34.759156187315455, Altitude: 0
